In [1]:
import numpy as np

from pyboolnet.external.bnet2primes import bnet_text2primes
from pyboolnet.state_transition_graphs import primes2stg

from transition_matrix import get_transition_matrix, get_uniform_matrix
from scc_dags import get_scc_dag, get_ordered_states, get_attractor_states
from matrix_operations import nsquare, compress_matrix, expand_matrix, get_confusion_matrix
from grouping import sd_grouping, null_grouping
from reachability import get_convergence_matrix
from basins import get_strong_basins

In [2]:
DEBUG = True

np.set_printoptions(linewidth=1000, precision=3, suppress=True)

In [3]:
bnet = """
A, A | B & C
B, B & !C
C, B & !C | !C & !D | !B & C & D
D, !A & !B & !C & !D | !A & C & D
"""

update = "asynchronous"

primes = bnet_text2primes(bnet)
primes = {key: primes[key] for key in sorted(primes)}
stg = primes2stg(primes, update)

scc_dag = get_scc_dag(stg)
scc_indices = get_ordered_states(scc_dag, as_indices=True, DEBUG=DEBUG)
attractor_indices = get_attractor_states(scc_dag, as_indices=True, DEBUG=DEBUG)

In [4]:
T = get_transition_matrix(stg, update=update, DEBUG=DEBUG)
T_inf = nsquare(T, 20, DEBUG=DEBUG)
T_convergence, _, _ = get_convergence_matrix(T_inf, scc_indices, attractor_indices, DEBUG=DEBUG)

print(T_convergence)

[[0.  0.  1. ]
 [0.3 0.2 0.5]
 [0.1 0.4 0.5]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.5 0.  0.5]
 [0.5 0.  0.5]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]]


In [5]:
T_strong_basin = get_strong_basins(T_convergence, DEBUG=DEBUG)

print(T_strong_basin)

[[1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]


In [6]:
sd_indices = sd_grouping(bnet, DEBUG=DEBUG)
Tsd = compress_matrix(T, sd_indices, DEBUG=DEBUG)
Tsd_inf = nsquare(Tsd, 20, DEBUG=DEBUG)
Tsd_inf_expanded = expand_matrix(Tsd_inf, sd_indices, DEBUG=DEBUG)
Tsd_convergence, _, _ = get_convergence_matrix(Tsd_inf_expanded, scc_indices, attractor_indices, DEBUG=DEBUG)

print(Tsd_convergence)

sd_strong_basin = get_strong_basins(Tsd_convergence, DEBUG=DEBUG)

print(sd_strong_basin)

TP, FP, TN, FN = get_confusion_matrix(T_strong_basin, sd_strong_basin, partial=True, DEBUG=DEBUG)

print(TP, FP, TN, FN)


[[0.   0.   1.  ]
 [0.25 0.25 0.5 ]
 [0.25 0.25 0.5 ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.25 0.25 0.5 ]
 [0.25 0.25 0.5 ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]]
[[1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]
6 0 4 0


In [7]:
null_indices = null_grouping(bnet, DEBUG=DEBUG)
Tnull = compress_matrix(T, null_indices, DEBUG=DEBUG)
Tnull_inf = nsquare(Tnull, 20, DEBUG=DEBUG)
Tnull_inf_expanded = expand_matrix(Tnull_inf, null_indices, DEBUG=DEBUG)
Tnull_convergence, _, _ = get_convergence_matrix(Tnull_inf_expanded, scc_indices, attractor_indices, DEBUG=DEBUG)

print(Tnull_convergence)

null_strong_basin = get_strong_basins(Tnull_convergence, DEBUG=DEBUG)

print(null_strong_basin)

TP, FP, TN, FN = get_confusion_matrix(T_strong_basin, null_strong_basin, partial=True, DEBUG=DEBUG)

print(TP, FP, TN, FN)

[[0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]
 [0.   0.25 0.75]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
0 0 4 6


In [8]:
Tuniform = get_uniform_matrix(T.shape[0])
Tuniform_convergence, _, _ = get_convergence_matrix(Tuniform, scc_indices, attractor_indices, DEBUG=DEBUG)

print(Tuniform_convergence)

uniform_strong_basin = get_strong_basins(Tuniform_convergence, DEBUG=DEBUG)

print(uniform_strong_basin)

TP, FP, TN, FN = get_confusion_matrix(T_strong_basin, uniform_strong_basin, partial=True, DEBUG=DEBUG)

print(TP, FP, TN, FN)

[[0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]
 [0.188 0.062 0.125]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
0 0 4 6
